# Exporting Models to ONNX

## Loading the Model

In [ ]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")

## Exporting to ONNX

In [ ]:
from archai.common.file_utils import calculate_onnx_model_size
from archai.onnx.export import export_to_onnx

onnx_model_path = "model.onnx"
onnx_config = export_to_onnx(
        model,
        onnx_model_path,
        task="causal-lm",
        use_past=True,
        share_weights=True,
        opset=11,
        atol=1e-4,
    )
print(f"Model: {calculate_onnx_model_size(onnx_model_path)}MB")

## Post-Export Optimization

In [ ]:
from archai.onnx.optimization import optimize_onnx

ort_model_path = optimize_onnx(onnx_model_path, onnx_config, opt_level=1)
print(f"Model-OPT: {calculate_onnx_model_size(ort_model_path)}MB")

## Post-Training Quantization (PTQ)

In [ ]:
from archai.quantization.ptq import dynamic_quantization_onnx

qnt_model_path = dynamic_quantization_onnx(ort_model_path)
print(f"Model-QNT: {calculate_onnx_model_size(qnt_model_path)}MB")